# Final project: Board game recommender

In [1208]:
# import libraries
import pandas as pd
import numpy as np
import scipy.stats as st

import requests
import pickle
from collections import Counter

pd.set_option("display.max_columns", 0) #no limit to cols we want to see

In [1209]:
# import datasets from boardgamegeek
categories = pd.read_csv('./csv_files/bgg_Category.csv')
game_families = pd.read_csv('./csv_files/bgg_GameFamily.csv')
game_items = pd.read_csv('./csv_files/bgg_GameItem.csv')
game_types = pd.read_csv('./csv_files/bgg_GameType.csv')
mechanics = pd.read_csv('./csv_files/bgg_Mechanic.csv')
authors = pd.read_csv('./csv_files/bgg_Person.csv')
publishers = pd.read_csv('./csv_files/bgg_Publisher.csv')

In [1210]:
game_items

,bgg_id,name,year,game_type,designer,artist,publisher,min_players,max_players,min_players_rec,max_players_rec,min_players_best,max_players_best,min_age,min_age_rec,min_time,max_time,category,mechanic,cooperative,compilation,compilation_of,family,implementation,integration,rank,num_votes,avg_rating,stddev_rating,bayes_rating,complexity,language_dependency,bga_id,dbpedia_id,luding_id,spielen_id,wikidata_id,wikipedia_id
0,1,Die Macher,1986.0,5497,1,"12517,4959","133,2,24883,2726,15108,39249,11652,5382,8147",3.0,5.0,4.0,5.0,5.0,5.0,14.0,14.027778,240.0,240.0,"1021,1026,1001","2916,2080,2012,2072,2040,2020",NaN,NaN,NaN,"10643,34116,91",NaN,NaN,369.0,5632.0,7.60139,1.57510,7.06367,4.3144,1.166667,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Dragonmaster,1981.0,5497,8384,12424,"64,20",3.0,4.0,3.0,4.0,3.0,4.0,12.0,NaN,30.0,30.0,"1002,1010",2009,NaN,NaN,NaN,"73592,7005,75454",2174,NaN,4490.0,584.0,6.65823,1.46752,5.76599,1.9636,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Samurai,1998.0,5497,2,11883,"17,133,267,29,7340,7335,41,2973,4617,1391,8291...",2.0,4.0,2.0,4.0,3.0,3.0,10.0,9.733333,30.0,60.0,"1009,1035","2080,2040,2026,2846,2004,2002",NaN,NaN,NaN,"64960,10634,70711,60111,42,28732",NaN,NaN,252.0,15863.0,7.45913,1.19279,7.22869,2.4784,1.006579,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Tal der Könige,1992.0,5497,8008,2277,37,2.0,4.0,2.0,4.0,2.0,4.0,12.0,NaN,60.0,60.0,1050,"2001,2080,2012,2004",NaN,NaN,NaN,"64229,6471,11505",NaN,NaN,5990.0,351.0,6.62296,1.23667,5.66660,2.6667,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Acquire,1964.0,5497,4,"12658,18317","92,5,4871,3082,858,2962,28072,5392,4668,38,462...",2.0,6.0,3.0,6.0,4.0,4.0,12.0,11.728571,90.0,90.0,"1021,1086","2040,2910,2900,2940,2005,2002,2874",NaN,NaN,NaN,"78494,4891",NaN,NaN,319.0,19494.0,7.33439,1.33603,7.12541,2.4985,1.095890,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113899,391565,Champions!,2023.0,NaN,"62310,62311,6736",NaN,"4384,1391",3.0,8.0,3.0,8.0,3.0,8.0,10.0,NaN,30.0,45.0,1030,2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
113900,391715,Leitin að stjörnunni,2022.0,NaN,"154290,123339,154289",NaN,54258,2.0,20.0,2.0,20.0,2.0,20.0,5.0,NaN,30.0,30.0,"1041,1064,1030,1027","2073,2038,2019",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
113901,391720,The String Railway Collection,2024.0,NaN,39436,113637,44392,2.0,5.0,2.0,5.0,2.0,5.0,10.0,NaN,20.0,45.0,"1086,1034","2011,2081,2007",NaN,1.0,"76674,100473",11331,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
113902,391834,SpellBook,2023.0,NaN,8347,45292,25842,1.0,4.0,1.0,4.0,1.0,4.0,12.0,NaN,45.0,45.0,1010,"2040,2004,2819",NaN,NaN,NaN,22184,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


BoardGameGeek is our major target for information regarding the board games. It is a community-run database that includes user rankings and recommendations based on number of players: time to beat/finish: category: mechanics of game: etc. Each major filter is divided by dataframe that converges/is IDd in `game_items`: and we will now asess them.

In [1211]:
# reviewing categories - 84
display(categories.head(30))

,bgg_id,name
0,1001,Political
1,1002,Card Game
2,1008,Nautical
3,1009,Abstract Strategy
4,1010,Fantasy
5,1011,Transportation
6,1013,Farming
7,1015,Civilization
8,1016,Science Fiction
9,1017,Dice


`Categories` show the genre of the game: or the theme that it will set in.

In [1212]:
# reviewing mechanics - 191
display(mechanics.head(30))

,bgg_id,name
0,2001,Action Points
1,2002,Tile Placement
2,2003,Rock-Paper-Scissors
3,2004,Set Collection
4,2005,Stock Holding
5,2007,Pick-up and Deliver
6,2008,Trading
7,2009,Trick-taking
8,2010,Crayon Rail System
9,2011,Modular Board


`Mechanics` deals with specific approach to the game that will be the main way of playing: or give insight to what tools might be needed to play a round.

In [1213]:
# reviewing game types - 12
game_types

,bgg_id,name
0,4415,Amiga
1,4420,Commodore 64
2,4664,War Game
3,4665,Children's Game
4,4666,Abstract Game
5,4667,Customizable
6,4781,Arcade
7,4976,Atari ST
8,5496,Thematic
9,5497,Strategy Game


`game_types` are a broader type of category: that can encapsulate the audience it is meant for.

In [1214]:
# reviewing game_families - 4828
game_families

,bgg_id,name
0,1,Admin: Test Family
1,2,Game: Carcassonne
2,3,Game: Catan
3,4,Series: The Chicken Family of Zoch
4,5,Game: Bohnanza
...,...,...
4823,80388,Series: Evolution of Airpower in World War 2
4824,80397,Series: Air War Book Games (Historic Wings)
4825,80399,Video Game Theme: Assassin's Creed
4826,80400,Video Game Theme: The Elder Scrolls


In [1215]:
# reviewing game_items - 113904
game_items.isna().sum()

bgg_id                      0
name                        0
year                     9685
game_type               88074
designer                18251
artist                  66032
publisher                  52
min_players              1938
max_players              5456
min_players_rec          1938
max_players_rec          5456
min_players_best         1938
max_players_best         5456
min_age                 23263
min_age_rec            112778
min_time                22562
max_time                22562
category                 2214
mechanic                16767
cooperative            107550
compilation            112989
compilation_of         112989
family                  34859
implementation         108039
integration            109784
rank                    89425
num_votes               27722
avg_rating              27722
stddev_rating           45407
bayes_rating            88970
complexity              65614
language_dependency    113050
bga_id                 113904
dbpedia_id

***

#### Fun fact counts before cleaning:

In [1216]:
# tHERE ARE 328 KINDS OF CHESS IN THIS COMMUNITY
chess_count = game_items[(game_items.name.str.contains('Chess')) | (game_items.name.str.contains(' Chess')) | (game_items.name.str.contains(' chess'))][['name','year']]

,name,year
85767,Speed Chess,NaN
85988,Warmaster Chess 2000: Variants,1998.0
86578,Chess Unbound,2018.0
86646,Imperial Chess,1993.0
87582,Gravity Chess,2013.0
87982,Super Chess,2018.0
88231,Story Time Chess,2019.0
88633,Frog Chess,2019.0
89245,Monkey King Chess,2004.0
89536,King Arthur's Vortex Chess,2020.0


In [1218]:
# inlfuenced by historic events, there are 105 games about wwii
wwii_count = game_items[(game_items.name.str.contains(' War II'))][['name','year']]

,name,year
44851,Combat Command: World War II Skirmish Rules fo...,2002.0
46430,Napoleon's War II: The Gates of Moscow,2011.0
47708,Kampfgruppe Normandy: Tabletop Wargame in Worl...,2011.0
49777,G.I. World War II Tactical Combat,2001.0
50783,Victory Decision: Advanced Wargaming Rules – W...,2011.0
51124,Baptism of Fire III: World War II Skirmish War...,2009.0
53022,Tactical Commander: Rules for World War II Ski...,1982.0
53023,Unit Organizations of World War II,1975.0
53126,World War II: Blitzkrieg 1940,2012.0
55474,World War II in Europe,2011.0


In [1221]:
# 13 games involving data as direct topic or mechanism
data_count = game_items[(game_items.name.str.contains('Data')) | (game_items.name.str.contains(' Data '))]

,bgg_id,name,year,game_type,designer,artist,publisher,min_players,max_players,min_players_rec,max_players_rec,min_players_best,max_players_best,min_age,min_age_rec,min_time,max_time,category,mechanic,cooperative,compilation,compilation_of,family,implementation,integration,rank,num_votes,avg_rating,stddev_rating,bayes_rating,complexity,language_dependency,bga_id,dbpedia_id,luding_id,spielen_id,wikidata_id,wikipedia_id
11104,12651,Dataflow,1978.0,NaN,48172,NaN,"5162,2834",2.0,4.0,2.0,4.0,2.0,4.0,12.0,NaN,NaN,NaN,NaN,"2041,2078",NaN,NaN,NaN,"73593,71023",NaN,NaN,NaN,5.0,6.20000,0.748331,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12858,14673,Data Finans,NaN,NaN,3,NaN,"404,3581",2.0,6.0,2.0,6.0,2.0,6.0,10.0,NaN,NaN,NaN,"1021,1072,1026","2012,2035,2008",NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0,4.48333,1.545060,NaN,1.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13525,15403,Data Boxing II,1976.0,NaN,4720,NaN,"5036,1838",1.0,2.0,1.0,2.0,1.0,2.0,10.0,NaN,30.0,30.0,1038,"2072,2070",NaN,NaN,NaN,"5839,5786",NaN,NaN,NaN,12.0,7.50000,2.217360,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44016,83071,Data Fortress,NaN,NaN,NaN,9798,14123,2.0,2.0,2.0,2.0,2.0,2.0,14.0,NaN,30.0,30.0,1009,"2040,2016,2002",NaN,NaN,NaN,"8374,61979",NaN,NaN,NaN,2.0,7.00000,1.000000,NaN,2.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45060,89107,DataMan,1977.0,NaN,NaN,NaN,2271,1.0,4.0,1.0,4.0,1.0,4.0,10.0,NaN,NaN,NaN,"1041,1094,1072,1104,1016","2023,2047",1.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,8.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57242,141133,The Game of Data Processing,1983.0,NaN,NaN,NaN,24964,2.0,3.0,2.0,3.0,2.0,3.0,10.0,NaN,NaN,NaN,1094,2035,NaN,NaN,NaN,"73593,71023",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68786,184442,Trivial Pursuit: Marvel Cinematic Universe – D...,2013.0,NaN,NaN,NaN,"51,4852,634",2.0,24.0,2.0,24.0,2.0,24.0,12.0,NaN,90.0,90.0,1027,2072,NaN,NaN,NaN,"5338,101,17935",NaN,213711,20004.0,30.0,5.81733,1.845980,5.50161,1.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70400,192221,Backup Storage War: Exagrid Vs. EMC Data Domain,2016.0,NaN,NaN,NaN,3320,2.0,4.0,2.0,4.0,2.0,4.0,18.0,NaN,NaN,NaN,"1094,1031",2035,NaN,NaN,NaN,11505,NaN,NaN,NaN,1.0,1.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74421,213711,Trivial Pursuit: Marvel Cinematic Universe – D...,2016.0,NaN,NaN,NaN,"51,4852,634",2.0,24.0,2.0,24.0,2.0,24.0,12.0,NaN,90.0,90.0,1027,2072,NaN,NaN,NaN,"5338,101,17935",NaN,184442,NaN,11.0,5.99818,2.266760,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
77243,229370,Rescue Polar Bears: Data & Temperature,2017.0,5499,"4896,93757","96824,93830","9215,37659,33617,42145,9552",1.0,4.0,1.0,4.0,1.0,4.0,8.0,NaN,30.0,60.0,"1089,1084","2001,2023,2026,2960,2011,2007,2884,2015,2897",1.0,NaN,NaN,"7265,22135,12809,22811,76250",205544,NaN,3290.0,601.0,7.23747,1.187610,5.90820,2.3333,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1222]:
# the oldest game
oldie = game_items[game_items['year'] == game_items['year'].min()][['name','year']]

,name,year
2060,Senet,-3500.0


In [1223]:
# the longest game
longest_playtime = game_items[game_items['max_time'] == game_items['max_time'].max()][['name','year','min_time','max_time']]
longest_playtime
# to a 1051200hrs
# 43800 days

,name,year,min_time,max_time
104953,RAIN,2021.0,1.0,63072000.0


In [1224]:
# year counts
years = game_items[['year']]
years.value_counts().sort_values(ascending=False).head(60)

# top years per decade
top_year_decades = pd.DataFrame({'year':[2019,2009,1999,1986,1975,1969], 'count':[5087,2843,1125,899,605,318]})
top_year_decades

year  
2019.0    5087
2020.0    4926
2021.0    4693
2018.0    4692
2022.0    4436
2017.0    4362
2016.0    4209
2015.0    3898
2014.0    3663
2013.0    3179
2012.0    3141
2011.0    3028
2010.0    2878
2009.0    2843
2023.0    2597
2007.0    2552
2008.0    2538
2005.0    2448
2006.0    2419
2004.0    2064
2003.0    1742
2002.0    1637
2001.0    1403
2000.0    1299
1999.0    1125
1998.0    1093
1995.0     957
1997.0     919
1990.0     912
1996.0     903
1986.0     899
1992.0     897
1987.0     889
1994.0     880
1991.0     867
1985.0     855
1989.0     846
1993.0     827
1988.0     803
1984.0     777
1983.0     733
1981.0     623
1980.0     617
1975.0     605
1982.0     586
1979.0     568
1977.0     566
1978.0     521
1976.0     520
1974.0     433
1973.0     420
1970.0     366
1972.0     353
1969.0     318
1971.0     275
2024.0     261
1965.0     233
1968.0     223
1960.0     217
1967.0     217
dtype: int64

In [1226]:
# save all this in files before cleaning
# chess_count.to_csv('chess_count.csv')
# wwii_count.to_csv('wwii_count.csv')
# data_count.to_csv('data_count.csv')
# longest_playtime.to_csv('longest_playtime.csv')
# oldie.to_csv('oldie.csv')
# years.to_csv('years.csv')

### Columns description:
- `bgg_id`: unique ID per board game; int.
- `name`: name of game; object.
- `year`: year of release; float.
- `game_type`: in-site's rough partition of game genres. Curently there are:
    - Uncategorized
    - Abstract Strategy Games (like Chess or Go)
    - Customizable Games (CCGs, CMGs, LCGs, etc.)
    - Thematic Games (emphasis on narrative)
    - Family Games (fun for kids and adults)
    - Children's Party Games (best for younger kids)
    - Strategy Games (more complex games)
    - Wargames (conflict simulation, etc.)
- `designer`: 
- `artist`:
- `publisher`:
- `min_players`: minimum of players by manufacturer; float.
- `max_players`: maximum of players by manufacturer; float.
- `min_players_rec`: minimum of players by the community; float.
- `max_players_rec`: maximum of players by the community; float.
- `min_players_best`: best minimum of players by the community; float.
- `max_players_best`: best maximum of players by the community; float.
- `min_age`: required minimum age to play; float.
- `min_age_rec`: required minimum age to play recommended by community; float.
- `min_time`: minimum gameplay time, measured in minutes; float.
- `max_time`: maximum gameplay time, measured in minutes; float.
- `category`: additional grouping approach based on like subjects or similar characteristics; object.
- `mechanic`: metaphorical term referring to a functional aspect of a game; object.
- `cooperative`: ??? ; float.
- `compilation`: boolean confirming whether it's part of a compilation of games; float.
- `compilation_of`: name of compilation; obj.
- `family`: group of games from the same franchise; obj.
- `implementation`: ???
- `integration`: ???
- `rank`
- `num_votes`
- `avg_rating`
- `stddev_rating`
- `bayes_rating`
- `complexity`: measure of difficulty of the gameplay between 1 and 5; float. 
- `language_dependency`: measure of how much text and other language dependencies appear on a game's components or during gameplay between 1 and 5; float. 
- `bga_id`: id of the game in different site.
- `dbpedia_id`: id of the game in different site.
- `luding_id`: id of the game in different site.
- `spielen_id`: id of the game in different site.
- `wikidata_id`: id of the game in different site.
- `wikipedia_id`: id of the game in different site.

### Columns to keep for mvp:
- `bgg_id`:
- `name`:
- `year`:
- `designer`:
- `publisher`:
- `game_type`: in-site's rough partition of game genres (currently 8); object. *
- `min_time`: minimum gameplay time, measured in minutes; float. *
- `max_time`:
- `min_age`:
- `min_players`: minimum of players by manufacturer; float. *
- `max_players`: 
- `category`: additional grouping approach based on like subjects or similar characteristics; object. *
- `mechanic`: metaphorical term referring to a functional aspect of a game; object. *
- `complexity`: measure of difficulty of the gameplay between 1 and 5; float. *
- `language_dependency`: measure of how much text and other language dependencies appear on a game's components or during gameplay between 1 and 5; float. *
- `avg_rating`: TBD
- `rank`: TBD 
### Columns to drop:
- `artist`:
- `min_players_rec`: 
- `max_players_rec`: 
- `min_players_best`:
- `max_players_best`: 
- `min_age_rec`:
- `family`:
- `implementation`:
- `integration`:
- `cooperative`: 
- `compilation`: 
- `compilation_of`:
- `num_votes`:
- `stddev_rating`:
- `bayes_rating`:
- `bga_id`:
- `dbpedia_id`:
- `luding_id`:
- `spielen_id`:
- `wikidata_id`:
- `wikipedia_id`: 

In [1227]:
# drop cols that we don't need from game_items
game_items.drop([
    'artist', 'min_players_rec','max_players_rec', 'min_players_best', 'max_players_best', 
    'min_age_rec', 'cooperative', 'compilation', 'compilation_of', 'family',
    'implementation', 'integration', 'num_votes','stddev_rating', 'bayes_rating', 
    'bga_id', 'dbpedia_id', 'luding_id', 'spielen_id', 'wikidata_id', 'wikipedia_id'],
    axis = 1, inplace = True)

In [1228]:
game_items
game_items.isna().sum()

bgg_id                      0
name                        0
year                     9685
game_type               88074
designer                18251
publisher                  52
min_players              1938
max_players              5456
min_age                 23263
min_time                22562
max_time                22562
category                 2214
mechanic                16767
rank                    89425
avg_rating              27722
complexity              65614
language_dependency    113050
dtype: int64

In [1229]:
# assess nans in language_dep
test = round(game_items.language_dependency)
test.value_counts(dropna=False)

NaN    113050
1.0       400
4.0       149
2.0       146
3.0       144
5.0        15
Name: language_dependency, dtype: int64

In [1230]:
game_items[game_items.language_dependency.isna()]

,bgg_id,name,year,game_type,designer,publisher,min_players,max_players,min_age,min_time,max_time,category,mechanic,rank,avg_rating,complexity,language_dependency
1,2,Dragonmaster,1981.0,5497,8384,"64,20",3.0,4.0,12.0,30.0,30.0,"1002,1010",2009,4490.0,6.65823,1.9636,NaN
3,4,Tal der Könige,1992.0,5497,8008,37,2.0,4.0,12.0,60.0,60.0,1050,"2001,2080,2012,2004",5990.0,6.62296,2.6667,NaN
5,6,Mare Mediterraneum,1989.0,NaN,5,6,2.0,6.0,12.0,240.0,240.0,"1015,1008",2072,12146.0,6.54583,3.0000,NaN
6,7,Cathedral,1978.0,4666,1335,"6421,4089,672,786,155,148,327,93,17832,35425",2.0,2.0,8.0,20.0,20.0,"1009,1041,1029,1086","2043,2048,2060,2940,2002",2203.0,6.53356,1.7872,NaN
7,8,Lords of Creation,1993.0,NaN,6,"4675,7",2.0,5.0,12.0,120.0,120.0,"1015,1010",2011,10575.0,6.05493,2.4000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113899,391565,Champions!,2023.0,NaN,"62310,62311,6736","4384,1391",3.0,8.0,10.0,30.0,45.0,1030,2017,NaN,NaN,NaN,NaN
113900,391715,Leitin að stjörnunni,2022.0,NaN,"154290,123339,154289",54258,2.0,20.0,5.0,30.0,30.0,"1041,1064,1030,1027","2073,2038,2019",NaN,NaN,NaN,NaN
113901,391720,The String Railway Collection,2024.0,NaN,39436,44392,2.0,5.0,10.0,20.0,45.0,"1086,1034","2011,2081,2007",NaN,NaN,NaN,NaN
113902,391834,SpellBook,2023.0,NaN,8347,25842,1.0,4.0,12.0,45.0,45.0,1010,"2040,2004,2819",NaN,NaN,NaN,NaN


In [1231]:
# drop the lang_dep col
rankings = game_items[['name','rank']]
game_items.drop(columns=['language_dependency', 'rank', 'avg_rating', 'publisher', 'designer'], axis=1, inplace=True)

There are too many nans in `language_dependency` to be useful, so we are dropping it. We are also dropping `rank` and `avg_rating` due to having too many nans and not useful to our recommender, but we'll add `rank` again later for visualization.

In [1232]:
# assess nans in complexity
test = round(game_items.complexity)
test.value_counts(dropna=False)

NaN    65614
2.0    20134
1.0    17751
3.0     8062
4.0     2153
5.0      190
Name: complexity, dtype: int64

While there is a considerable amount of nans in `complexity`, the leftover amount is still good for a model so we will drop those nans and work with the data we have. Same with `category` and `mechanic`.

In [1233]:
# drop nans from complexity, mechanic and category
game_items.dropna(subset=['complexity', 'mechanic', 'category'], inplace=True)
game_items.reset_index(drop=True)

,bgg_id,name,year,game_type,min_players,max_players,min_age,min_time,max_time,category,mechanic,complexity
0,1,Die Macher,1986.0,5497,3.0,5.0,14.0,240.0,240.0,"1021,1026,1001","2916,2080,2012,2072,2040,2020",4.3144
1,2,Dragonmaster,1981.0,5497,3.0,4.0,12.0,30.0,30.0,"1002,1010",2009,1.9636
2,3,Samurai,1998.0,5497,2.0,4.0,10.0,30.0,60.0,"1009,1035","2080,2040,2026,2846,2004,2002",2.4784
3,4,Tal der Könige,1992.0,5497,2.0,4.0,12.0,60.0,60.0,1050,"2001,2080,2012,2004",2.6667
4,5,Acquire,1964.0,5497,2.0,6.0,12.0,90.0,90.0,"1021,1086","2040,2910,2900,2940,2005,2002,2874",2.4985
...,...,...,...,...,...,...,...,...,...,...,...,...
43087,389817,Route 66: Get Your Tricks,2023.0,NaN,2.0,2.0,12.0,25.0,30.0,"1002,1011,1097","2040,2047,2846,2876,2004,2009",2.0000
43088,389997,Bakhmut Meatgrinder: Verdun 2023,2023.0,NaN,1.0,6.0,14.0,60.0,120.0,"1023,1024,1069,1120,1019","2072,2875,2864,2026,2967,2837,2960,2958,2965,2...",2.0000
43089,390199,Space Privateer Command,2022.0,NaN,1.0,1.0,12.0,60.0,80.0,"1022,1017,1016",2819,2.0000
43090,390607,Great Kingdom,2023.0,NaN,2.0,2.0,8.0,20.0,20.0,1009,"2043,2940",2.0000


In [1234]:
# there are no board games with amiga, commodore, arcade, atari types so we will drop them from the game_types df
# drop values from game_type that are not present in game_items
game_types.drop(game_types[game_types.name.isin(['Amiga', 'Commodore 64', 'Arcade', 'Atari ST'])].index, inplace=True)
game_types.reset_index(drop=True, inplace=True)

In [1235]:
categories

,bgg_id,name
0,1001,Political
1,1002,Card Game
2,1008,Nautical
3,1009,Abstract Strategy
4,1010,Fantasy
...,...,...
79,2650,Aviation / Flight
80,2687,Fan Expansion
81,2710,Post-Napoleonic
82,2725,Pike and Shot


In [1236]:
mechanics

,bgg_id,name
0,2001,Action Points
1,2002,Tile Placement
2,2003,Rock-Paper-Scissors
3,2004,Set Collection
4,2005,Stock Holding
...,...,...
186,3100,Tags
187,3101,Ordering
188,3102,Questions and Answers
189,3103,Resource Queue


***

In [1237]:
# assess nans in game types
# according to the documentation, there is an official catg as Uncategorized - so we can assume that the nans are these uncategorised (we could call them other)
game_items.game_type.isna().sum() # 88074
game_items.game_type.fillna(6000, inplace=True)

In [1238]:
# add an unique id for uncategorized in game_type
uncategorized_row = {'bgg_id': 6000, 'name': 'Uncategorized'}
game_types.loc[len(game_types)] = uncategorized_row
game_types

,bgg_id,name
0,4664,War Game
1,4665,Children's Game
2,4666,Abstract Game
3,4667,Customizable
4,5496,Thematic
5,5497,Strategy Game
6,5498,Party Game
7,5499,Family Game
8,6000,Uncategorized


***

In [1239]:
# assess nans in years
# replace nans with unknown - good for fun fact maybe
game_items['year'].fillna(0, inplace=True)
game_items['year'] = pd.to_numeric(game_items['year'], errors='coerce', downcast='integer')
# game_items['year'] = int(float(game_items['year']))
game_items['year'] = game_items['year'].astype(str).replace(str(0),'Unknown')
game_items['year'].value_counts(dropna=False)

2019     2182
2018     2051
2017     1996
2020     1959
2016     1898
         ... 
1663        1
1200        1
1885        1
-1300       1
1917        1
Name: year, Length: 247, dtype: int64

In [1240]:
game_items[game_items['year'] == 'Unknown']

,bgg_id,name,year,game_type,min_players,max_players,min_age,min_time,max_time,category,mechanic,complexity
282,318,Looney Leo,Unknown,6000,NaN,NaN,NaN,NaN,NaN,1079,"2012,2008",1.0000
3192,3684,Dragons: The Classic Game,Unknown,4666,2.0,2.0,8.0,20.0,20.0,1009,"2043,2048",1.0000
3493,4031,Jungle,Unknown,4666,2.0,2.0,6.0,20.0,20.0,"1009,1089,1041","2676,2003,2940,2861",1.7632
3656,4213,Mao,Unknown,5498,2.0,20.0,13.0,5.0,60.0,"1002,1079,1045,1030","2040,3003,2047,2060",1.9714
3990,4599,Shades of Thought,Unknown,4665,2.0,2.0,6.0,10.0,10.0,"1032,1041",2060,1.0000
...,...,...,...,...,...,...,...,...,...,...,...,...
113111,383281,The Last Battle on Earth,Unknown,6000,2.0,4.0,14.0,60.0,180.0,"1069,1016,1086,1019","2072,2676,2026,2902,2047,2011,2947,2909,2003,2...",3.0000
113594,387483,Crimea: Conquest & Liberation,Unknown,6000,2.0,4.0,NaN,120.0,120.0,"1019,1049","2072,2026",4.0000
113623,387722,PROMISLAND,Unknown,6000,2.0,6.0,7.0,15.0,45.0,"1032,1002,1030,1090","2689,2960,2035",2.0000
113701,388396,Pékin-Paris,Unknown,6000,2.0,5.0,12.0,45.0,60.0,"1002,1020,1120,1031","2664,2040,2002",2.0000


In [1241]:
game_items.isna().sum()

bgg_id            0
name              0
year              0
game_type         0
min_players     202
max_players     703
min_age        5655
min_time       3838
max_time       3838
category          0
mechanic          0
complexity        0
dtype: int64

In [1242]:
game_items

,bgg_id,name,year,game_type,min_players,max_players,min_age,min_time,max_time,category,mechanic,complexity
0,1,Die Macher,1986,5497,3.0,5.0,14.0,240.0,240.0,"1021,1026,1001","2916,2080,2012,2072,2040,2020",4.3144
1,2,Dragonmaster,1981,5497,3.0,4.0,12.0,30.0,30.0,"1002,1010",2009,1.9636
2,3,Samurai,1998,5497,2.0,4.0,10.0,30.0,60.0,"1009,1035","2080,2040,2026,2846,2004,2002",2.4784
3,4,Tal der Könige,1992,5497,2.0,4.0,12.0,60.0,60.0,1050,"2001,2080,2012,2004",2.6667
4,5,Acquire,1964,5497,2.0,6.0,12.0,90.0,90.0,"1021,1086","2040,2910,2900,2940,2005,2002,2874",2.4985
...,...,...,...,...,...,...,...,...,...,...,...,...
113841,389817,Route 66: Get Your Tricks,2023,6000,2.0,2.0,12.0,25.0,30.0,"1002,1011,1097","2040,2047,2846,2876,2004,2009",2.0000
113848,389997,Bakhmut Meatgrinder: Verdun 2023,2023,6000,1.0,6.0,14.0,60.0,120.0,"1023,1024,1069,1120,1019","2072,2875,2864,2026,2967,2837,2960,2958,2965,2...",2.0000
113855,390199,Space Privateer Command,2022,6000,1.0,1.0,12.0,60.0,80.0,"1022,1017,1016",2819,2.0000
113876,390607,Great Kingdom,2023,6000,2.0,2.0,8.0,20.0,20.0,1009,"2043,2940",2.0000


In [1243]:
# assess nans in min ad nmax time, players, and min age
print('mode of min players: ', game_items.min_players.mode()[0]) # 2
print('mode of max players: ', game_items.max_players.mode()[0]) # 4
print('mode of min time: ', game_items.min_time.mode()[0]) # 30
print('mean of min time: ', game_items.min_time.mean()) # 61
print('median of min time: ', game_items.min_time.median()) # 30
print('mean of max time: ', game_items.max_time.mean())# 90
print('mode of min age: ', game_items.min_age.mode()[0])# 12
print('mean of min age: ', game_items.min_age.mean())# 9
print('min of min age: ', game_items.min_age.min())# 1 lol

mode of min players:  2.0
mode of max players:  4.0
mode of min time:  30.0
mean of min time:  61.15942324349111
median of min time:  30.0
mean of max time:  92.62296835991236
mode of min age:  12.0
mean of min age:  9.99121190266314
min of min age:  1.0


In [1244]:
# replace: 
# min and max players with mode
# min and max time with mode
# min age with mean
game_items.min_players.fillna(game_items.min_players.mode()[0], inplace=True)
game_items.max_players.fillna(game_items.max_players.mode()[0], inplace=True)
game_items.min_time.fillna(game_items.min_time.mode()[0], inplace=True)
game_items.max_time.fillna(round(game_items.max_time.mean()), inplace=True)
game_items.min_age.fillna(round(game_items.min_age.mean()), inplace=True)

In [1245]:
# round numericals
game_items[['min_players','max_players','min_time','max_time','min_age','complexity']] = game_items[['min_players','max_players','min_time','max_time','min_age','complexity']].astype(int)

**Wednesday's steps:**
1. Drop language_dependency
2. Drop nans in complexity
3. Drop values from game_type that are not in game_items
4. Replace id refs with values
5. Cry a bit maybe
9. Drop mechanic nans
6. Replace min and max players with mode
7. Replace min age with mean
8. Replace min and max time with mode and mean
10. HOPEFULLY be ready to start treating it

In [1246]:
# round complexity numbers
game_items['complexity'] = round(game_items['complexity'])
game_items

,bgg_id,name,year,game_type,min_players,max_players,min_age,min_time,max_time,category,mechanic,complexity
0,1,Die Macher,1986,5497,3,5,14,240,240,"1021,1026,1001","2916,2080,2012,2072,2040,2020",4
1,2,Dragonmaster,1981,5497,3,4,12,30,30,"1002,1010",2009,1
2,3,Samurai,1998,5497,2,4,10,30,60,"1009,1035","2080,2040,2026,2846,2004,2002",2
3,4,Tal der Könige,1992,5497,2,4,12,60,60,1050,"2001,2080,2012,2004",2
4,5,Acquire,1964,5497,2,6,12,90,90,"1021,1086","2040,2910,2900,2940,2005,2002,2874",2
...,...,...,...,...,...,...,...,...,...,...,...,...
113841,389817,Route 66: Get Your Tricks,2023,6000,2,2,12,25,30,"1002,1011,1097","2040,2047,2846,2876,2004,2009",2
113848,389997,Bakhmut Meatgrinder: Verdun 2023,2023,6000,1,6,14,60,120,"1023,1024,1069,1120,1019","2072,2875,2864,2026,2967,2837,2960,2958,2965,2...",2
113855,390199,Space Privateer Command,2022,6000,1,1,12,60,80,"1022,1017,1016",2819,2
113876,390607,Great Kingdom,2023,6000,2,2,8,20,20,1009,"2043,2940",2


In [1247]:
game_items.isna().sum()

bgg_id         0
name           0
year           0
game_type      0
min_players    0
max_players    0
min_age        0
min_time       0
max_time       0
category       0
mechanic       0
complexity     0
dtype: int64

In [1248]:
game_items.reset_index(drop=True, inplace=True)

***

In [1249]:
# fill category
c = categories.astype(str).set_index('bgg_id')['name']
c
game_items['category'] = (
    game_items['category'].str.split(',')
    .explode().map(c).groupby(level=0).agg(','.join)
)
game_items['category']


0                           Economic,Negotiation,Political
1                                        Card Game,Fantasy
2                               Abstract Strategy,Medieval
3                                                  Ancient
4                              Economic,Territory Building
                               ...                        
43087                      Card Game,Transportation,Travel
43088    Bluffing,Horror,Modern Warfare,Print & Play,Wa...
43089                       Adventure,Dice,Science Fiction
43090                                    Abstract Strategy
43091    Aviation / Flight,Economic,Industry / Manufact...
Name: category, Length: 43092, dtype: object

In [1250]:
# break down lists for top 20 values in category

res = []
for i, elem in enumerate(game_items["category"].to_list()):
    res.extend([''.join(filter(str.isalnum, e)).lower() for e in elem.split(sep=",")]) 


# make a new df to read better
df2 = pd.DataFrame()
df2 = df2.assign(listedWords=Counter(res).keys(),    # list each unique elements
                occurences=Counter(res).values())   # list occurences
df2.sort_values(by='occurences', ascending=False).head(20)

,listedWords,occurences
3,cardgame,11875
28,wargame,8660
4,fantasy,4560
11,childrensgame,4365
20,dice,4139
5,abstractstrategy,3598
40,partygame,3575
18,sciencefiction,3129
22,printplay,2953
21,fighting,2861


In [1251]:
df2['listedWords'] = df2['listedWords'].str.capitalize()
df2['listedWords'] = df2['listedWords'].apply(lambda x: categories.loc[categories['name'].str[:5] == x[:5], 'name'].values[0] if any(categories['name'].str[:5] == x[:5]) else x)
df2

,listedWords,occurences
0,Economic,2776
1,Negotiation,1199
2,Political,995
3,Cardgame,11875
4,Fantasy,4560
...,...,...
79,Mature / Adult,360
80,Vietnam War,161
81,Korean War,66
82,Expansion for Base-game,71


In [1252]:
# fill mechanic
m = mechanics.astype(str).set_index('bgg_id')['name']
game_items['mechanic'] = (
    game_items['mechanic'].str.split(',')
    .explode().map(m).groupby(level=0).agg(','.join)
)
game_items['mechanic']

0        Alliances,Area Majority / Influence,Auction/Bi...
1                                             Trick-taking
2        Area Majority / Influence,Hand Management,Hexa...
3        Action Points,Area Majority / Influence,Auctio...
4        Hand Management,Investment,Market,Square Grid,...
                               ...                        
43087    Hand Management,Memory,Once-Per-Game Abilities...
43088    Dice Rolling,End Game Bonuses,Force Commitment...
43089                                Solo / Solitaire Game
43090                                Enclosure,Square Grid
43091    Action Points,Connections,End Game Bonuses,Han...
Name: mechanic, Length: 43092, dtype: object

In [1253]:
# break down lists for top 20 values in mechanics

res = []
for i, elem in enumerate(game_items["mechanic"].to_list()):
    res.extend([''.join(filter(str.isalnum, e)).lower() for e in elem.split(sep=",")]) 


# make a new df to read better
df3 = pd.DataFrame()
df3 = df3.assign(listedWords=Counter(res).keys(),    # list each unique elements
            occurences=Counter(res).values())   # list occurences
df3.sort_values(by='occurences', ascending=False).head(20)

,listedWords,occurences
3,dicerolling,12926
4,handmanagement,7719
9,setcollection,5631
7,hexagongrid,5060
50,simulation,4420
35,rollspinandmove,4313
36,variableplayerpowers,4256
22,opendrafting,3877
10,tileplacement,3172
39,gridmovement,3104


In [1269]:
df3[df3.name == 'Drawing']

,name,occurences
169,Drawing,21


In [1254]:
df3['listedWords'] = df3['listedWords'].str.capitalize()
df3['listedWords'] = df3['listedWords'].apply(lambda x: mechanics.loc[mechanics['name'].str[:3] == x[:3], 'name'].values[0] if any(mechanics['name'].str[:3] == x[:3]) else x)
df3

,listedWords,occurences
0,Alliances,97
1,Area-Impulse,2624
2,Auction/Bidding,1811
3,Dice Rolling,12926
4,Hand Management,7719
...,...,...
189,Neighbor Scope,5
190,Bids As Wagers,3
191,Legacy Game,53
192,Auction/Bidding,3


In [1255]:
# fill game_type
gt = game_types.astype(str).set_index('bgg_id')['name']
game_items['game_type'] = game_items['game_type'].astype(str)
game_items['game_type'] = (
    game_items['game_type'].str.split(',')
    .explode().map(gt).astype(str).groupby(level=0).agg(','.join)
)
game_items['game_type']

0        Strategy Game
1        Strategy Game
2        Strategy Game
3        Strategy Game
4        Strategy Game
             ...      
43087    Uncategorized
43088    Uncategorized
43089    Uncategorized
43090    Uncategorized
43091    Uncategorized
Name: game_type, Length: 43092, dtype: object

In [1256]:
# break down lists for top 20 values in game type

res = []
for i, elem in enumerate(game_items["game_type"].to_list()):
    res.extend([''.join(filter(str.isalnum, e)).lower() for e in elem.split(sep=",")]) 


# make a new df to read better
df4 = pd.DataFrame()
df4 = df4.assign(listedWords=Counter(res).keys(),    # list each unique elements
                occurences=Counter(res).values())   # list occurences
df4.sort_values(by='occurences', ascending=False).head(10)

,listedWords,occurences
1,uncategorized,25301
6,wargame,7247
3,familygame,2883
0,strategygame,2723
2,abstractgame,2355
8,childrensgame,1772
4,thematic,1447
7,partygame,759
5,customizable,372
9,nan,4


In [1257]:
# Use a lambda function to replace values in df1 based on matching first 5 letters
df4['listedWords'] = df4['listedWords'].str.capitalize()
df4['listedWords'] = df4['listedWords'].apply(lambda x: game_types.loc[game_types['name'].str[:5] == x[:5], 'name'].values[0] if any(game_types['name'].str[:5] == x[:5]) else x)
df4

,listedWords,occurences
0,Strategy Game,2723
1,Uncategorized,25301
2,Abstract Game,2355
3,Family Game,2883
4,Thematic,1447
5,Customizable,372
6,Wargame,7247
7,Party Game,759
8,Children's Game,1772
9,Nan,4


In [1258]:
game_items

,bgg_id,name,year,game_type,min_players,max_players,min_age,min_time,max_time,category,mechanic,complexity
0,1,Die Macher,1986,Strategy Game,3,5,14,240,240,"Economic,Negotiation,Political","Alliances,Area Majority / Influence,Auction/Bi...",4
1,2,Dragonmaster,1981,Strategy Game,3,4,12,30,30,"Card Game,Fantasy",Trick-taking,1
2,3,Samurai,1998,Strategy Game,2,4,10,30,60,"Abstract Strategy,Medieval","Area Majority / Influence,Hand Management,Hexa...",2
3,4,Tal der Könige,1992,Strategy Game,2,4,12,60,60,Ancient,"Action Points,Area Majority / Influence,Auctio...",2
4,5,Acquire,1964,Strategy Game,2,6,12,90,90,"Economic,Territory Building","Hand Management,Investment,Market,Square Grid,...",2
...,...,...,...,...,...,...,...,...,...,...,...,...
43087,389817,Route 66: Get Your Tricks,2023,Uncategorized,2,2,12,25,30,"Card Game,Transportation,Travel","Hand Management,Memory,Once-Per-Game Abilities...",2
43088,389997,Bakhmut Meatgrinder: Verdun 2023,2023,Uncategorized,1,6,14,60,120,"Bluffing,Horror,Modern Warfare,Print & Play,Wa...","Dice Rolling,End Game Bonuses,Force Commitment...",2
43089,390199,Space Privateer Command,2022,Uncategorized,1,1,12,60,80,"Adventure,Dice,Science Fiction",Solo / Solitaire Game,2
43090,390607,Great Kingdom,2023,Uncategorized,2,2,8,20,20,Abstract Strategy,"Enclosure,Square Grid",2


In [1259]:
# review rankings
# out of 113904, only 24479 are ranked
rankings.dropna(subset=['rank'], inplace=True)
rankings.reset_index(drop=True)
rankings

/var/folders/nn/t4c1n2z11_g_hd91q77nxzb80000gn/T/ipykernel_82076/3853810171.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rankings.dropna(subset=['rank'], inplace=True)


,name,rank
0,Die Macher,369.0
1,Dragonmaster,4490.0
2,Samurai,252.0
3,Tal der Könige,5990.0
4,Acquire,319.0
...,...,...
112485,Fiction,11444.0
112579,Cloudy Kingdom,15122.0
112666,Planet Trade,16635.0
112762,Deadly Dowagers,11644.0


In [1260]:
# rename dfs occurences cols for better reading
df2.rename(columns={'listedWords':'name'}, inplace=True)
df3.rename(columns={'listedWords':'name'}, inplace=True)
df4.rename(columns={'listedWords':'name'}, inplace=True)

In [1261]:
# save datasets for future use
# game_items.to_pickle('board_games.pkl')

# save data for visualization
# game_items.to_csv('board_games_clean.csv')
# rankings.to_csv('rankings.csv')
# df2.to_csv('categories_occurences.csv')
# df3.to_csv('mechanics_occurences.csv')
# df4.to_csv('gametypes_occurences.csv')

***

#### Test webscraping

In [1262]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.buzzfeed.com/jamesgrebey/mind-boggling-facts-about-board-games'
response = requests.get(url)
response.status_code

200

In [1263]:
# make that good soup
soup = BeautifulSoup(response.text, 'html.parser')

In [1264]:
# find all ps
text = []
# print all content
for p in soup.find_all('p'):
    print(p.get_text())
    text.append(p.get_text())

Reading these won't be a trivial pursuit.
BuzzFeed Staff
1. Original paper versions of Battleship included land areas in addition to the water.
2. Battleship was one of the first games to be made into a computer game in 1979. 
3. The inventor of Boggle, Allan Turoff, was married in FAO Schwarz's dollhouse department.
4. In order to prevent Boggle players from using a certain swear word, the letters F and K only appear once on the same cube, making it impossible for them to both be played at the same time.
5. Candy Land was at the center of one of the first disputes over domain names on the web, as in the mid-'90s candyland.com was a porn site. Hasbro successfully sued for control of the domain.
6. Candy Land was invented by a retired schoolteacher while she was recovering from polio.
7. The phrase "back to square one" might have been inspired by Chutes and Ladders.
8. The player who goes first in Connect Four can win 100% of the time.
9. A traditional Connect Four board has 4,531,985,2

In [1265]:
# turn it into a df
fun_facts = pd.DataFrame({'fun_fact': text})
fun_facts

,fun_fact
0,Reading these won't be a trivial pursuit.
1,BuzzFeed Staff
2,1. Original paper versions of Battleship inclu...
3,2. Battleship was one of the first games to be...
4,"3. The inventor of Boggle, Allan Turoff, was m..."
5,4. In order to prevent Boggle players from usi...
6,5. Candy Land was at the center of one of the ...
7,6. Candy Land was invented by a retired school...
8,"7. The phrase ""back to square one"" might have ..."
9,8. The player who goes first in Connect Four c...
